In [1]:
import os
import sys
import numpy
import random
from time import sleep
from copy import deepcopy
import tensorflow.keras as keras
from keras import layers
from itertools import count
from gui.puzzle_class import Twisty_Puzzle
from gui.ai_modules.ai_data_preparation import state_for_ai
from gui.ai_modules.ai_puzzle_class import puzzle_ai
from gui.ai_modules.twisty_puzzle_model import *

def action_num (puzzle, action):
    for i, act in zip(count(), puzzle.moves.keys()):
        if action == act :
            return i

def action_from_num (puzzle, num):
    list(puzzle.moves.values())[num]

def prepare_data (puzzle, q_table):
    training_table = dict()
    output_template = [0 for _ in range(len(puzzle.moves))]
    for state_action , value in q_table.items() :
        (state , action) = state_action
        state += tuple(state_for_ai(puzzle.SOLVED_STATE)[0])
        inner = training_table.get(state) or output_template.copy()
        inner[action_num(puzzle, action)] = value
        training_table[state] = inner
    return list(training_table.keys()) , list(training_table.values())

def import_q_table (filename="floppy_q_table.txt"):
    q_table = dict()
    with open(filename, "r") as file:
        q_table = eval (file.read())
    return q_table

<IPython.core.display.Javascript object>

In [2]:
def initialize_nn (puzzle):
    actions = puzzle.moves
    solved_state = state_for_ai(puzzle.SOLVED_STATE)[0]
    input_size = len(solved_state)*2
    output_size = len(actions)
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_size,)))
    model.add(layers.Dense((input_size+output_size)/2))
    model.add(layers.Dense(output_size))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [3]:
def train_nn (model, samples, labels, epochs=100, batch_size=30):
    return model.fit(samples, labels, epochs=epochs, batch_size=batch_size,
                     use_multiprocessing=True, verbose=False)

In [4]:
def nn_get_greedy_move (model, state, solved_state, actions):
    state_data = tuple(state+solved_state)
    predictions = list(model.predict([state_data], use_multiprocessing=True)[0])
    max_value = None
    moves = []
    for move, value in enumerate(predictions):
        if (not max_value) or value > max_value:
            moves = [move]
            max_value = value
        elif value == max_value:
            moves.append(move)
    return actions[random.choice(moves)]

def nn_get_epsilon_greedy_move (model, state, solved_state, actions, epsilon=0.5):
    if random.random() > epsilon:
        return nn_get_greedy_move(model, state, solved_state, actions)
    else:
        return random.choice(list(puzzle.moves.values()))

def nn_solve_puzzle (model, state, solved_state, actions, max_moves, epsilon=0):
    moves = []
    while max_moves >= 0:
        if state == solved_state:
            return moves, True
        else :
            move = nn_get_epsilon_greedy_move(model, state, solved_state, list(actions.keys()), epsilon=epsilon)
            moves.append(move)
            state = perform_action(state, actions[move])
            max_moves -= 1
    return moves, False

In [5]:
puzzle = Twisty_Puzzle()
puzzle.load_puzzle("floppy_cube")

## {{{ http://code.activestate.com/recipes/577058/ (r2)
def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.

    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).

    The "answer" return value is one of "yes" or "no".
    """
    valid = {"yes":True,   "y":True,  "ye":True,
             "no":False,     "n":False}
    if default == None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while 1:
        sys.stdout.write(question + prompt)
        choice = input().lower()
        if default is not None and choice == '':
            return default
        elif choice in valid.keys():
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "\
                             "(or 'y' or 'n').\n")
## end of http://code.activestate.com/recipes/577058/ }}}

model = None
if query_yes_no("Load the existing neural network?"):
    model = keras.models.load_model("floppy_cube.nn")
else:
    q_table = import_q_table()
    inputs, outputs = prepare_data(puzzle, q_table)
    model = initialize_nn(puzzle)
    history = train_nn(model, inputs, outputs, epochs=1000, batch_size=100)

    model.save("floppy_cube.nn")

solved_state = state_for_ai(puzzle.SOLVED_STATE)[0]
state = deepcopy(solved_state)
scramble_moves = scramble(state, puzzle.moves, max_moves=30)
start_state = deepcopy(state)

print("solved state:", solved_state)
print("start:       ", state)
(solution, solved) = nn_solve_puzzle(model, state, solved_state, puzzle.moves, 60)
print("end:         ", state)
print(solved)
print(solution)

for move in scramble_moves:
    puzzle.perform_move(move)
for move in solution:
    puzzle.perform_move(move)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Load the existing neural network? [Y/n] 
solved state: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]
start:        [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 2, 2, 2, 5, 3, 5, 4, 4, 4, 3, 5, 3]
end:          [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 2, 2, 2, 3, 3, 5, 4, 4, 4, 3, 5, 5]
False
['f', 'b', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']
